In [ ]:
# import all the things!
# yeah I know I don't need all this, but just incase...

import pandas as pd
import numpy as np
import os
%pwd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, ExtraTreesRegressor, ExtraTreesClassifier, BaggingRegressor,BaggingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor, GradientBoostingRegressor ,GradientBoostingClassifier
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
import pydot
from IPython.display import Image
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.preprocessing import StandardScaler

In [ ]:
# need to import df csv

df = pd.read_csv('gender_included_df.csv', encoding='utf-8')

In [ ]:
df.head()

In [ ]:
# create new df reducing to column for further preprocessing and my y-column
# now here I may want to make another df and include 'location_id', 'word_count' 
# I feel 'word_count' may be a huge impact (female characters have lower count than male--EDA)

df_reduced = df.copy()[['normalized_text','gender']]

df_expanded = df.copy()[['location_id', 'word_count', 'normalized_text', 'gender']]

In [ ]:
df_reduced.dropna(inplace=True)
df_reduced.head()

In [ ]:
df_expanded.dropna(inplace=True)
df_expanded.head()

In [ ]:
df_reduced['gender'].value_counts()

In [ ]:
# get baseline no. of female / no. of total rows

np.mean(df_reduced['gender'])

In [ ]:
df_expanded['gender'].value_counts()

In [ ]:
# get baseline no. of female / no. of total rows

np.mean(df_expanded['gender'])

In [ ]:
# need to pull out dependent variable (y)

y = LabelEncoder().fit_transform(df_reduced['gender'].values)

In [ ]:
# need to pull out dependent variable (y)

y = LabelEncoder().fit_transform(df_expanded['gender'].values)

In [ ]:
# set up train test split
# do on reduced first then go back to expanded if time

x_train, x_test, y_train, y_test = train_test_split(df_reduced, y)

In [ ]:
# check shape of train test split

print x_train.shape
print x_test.shape
print y_train.shape
print y_test.shape

In [ ]:
# need to run train test split through tfidf to fit data and then transform it and transform test data off of learned train data

tfidf = TfidfVectorizer(stop_words='english',
                       ngram_range=(1,3))
tfidf.fit(x_train)
transformed = tfidf.transform(x_train)
truncatedsvd.fit(transformed)
x_train_transformed = truncatedsvd.transform(transformed)

x_test_transformed = tfidf.transform(x_test)
x_test_transformed = truncatedsvd.transform(x_test_transformed)
# random forest without params
rf = RandomForestClassifier()
rf.fit(x_train_transformed, y_train)

predictions = rf.predict(x_test_transformed)
rf.score(x_test_transformed, y_test)

In [ ]:
confusion = pd.DataFrame(confusion_matrix(y_test, predictions), columns=['predicted_f', 'predicted_m'], index=['is_f', 'is_m'])
confusion

In [ ]:
print "Classification Report:\n", classification_report(y_test, predictions)

In [ ]:
## plots ROC curve
def plot_roc(x_test_transformed, y_test, model):
    fpr, tpr, _ = roc_curve(y_test, model.predict_proba(x_test)[:,1])
    roc_auc = auc(fpr, tpr)
    print 'ROC AUC: %0.2f' % roc_auc
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()
    
plot_roc(x_test_transformed, y_test, rf)    

In [ ]:
# taking a look at predictions to visualize where to alter probability cutoff

predict = rf.predict_proba(x_test_transformed)

plt.hist(predict[:,1], bins=40);

In [ ]:
## plots the distribution of prediction probabilities for the 1 class
def plot_proba(x_test, model):
    predict = model.predict_proba(x_test)
    plt.hist(predict[:,1])
    plt.ylabel('Frequency')
    plt.xlabel('Class 1 Probability Cutoff')
    plt.show()
    
plot_proba(x_test_transformed, rf)

In [ ]:
## calculates new scores based on inputed probability, then plots new model scores based on the range of cutoffs

def get_predictions(model, x_test, y_test, probability):
    predict = model.predict_proba(x_test)
    new_predictions = []
    for i in predict:
        if i[1] >= probability:
            new_predictions.append(1)
        else:
            new_predictions.append(0)
    return new_predictions
    
def find_optimal_probability(model, x_test, y_test):
    probabilities = range(1,101)
    new_probabilities = []
    for i in probabilities:
        new_probabilities.append(float(i)/100)
    range_of_scores = []
    for i in new_probabilities:
        new_preds = get_predictions(model, x_test, y_test, i)
        cmatrix = confusion_matrix(y_test, new_preds)
        new_model_score = (cmatrix[0,0] + cmatrix[1,1])/float(np.sum(cmatrix))
        range_of_scores.append(new_model_score)
    plt.plot(new_probabilities, range_of_scores)
    plt.xlabel("Probability Cutoff")
    plt.ylabel("Model Score")
    plt.show()

In [ ]:
# need to change probability

get_predictions(rf, x_test_transformed, probability)